# Du Chaos à MLflow : Un Parcours Guidé

## Objectif du Workshop

Vous venez d'expérimenter le **notebook chaotique** (`01_messy_notebook.ipynb`). Vous avez probablement remarqué :

- "Attends, c'était lequel le meilleur modèle déjà?"
- Métriques éparpillées dans les cellules, difficiles à comparer
- Aucun moyen de reproduire exactement les résultats
- Tracking manuel dans des tableaux markdown (que vous avez oublié de mettre à jour)

**Dans ce notebook, nous allons corriger tout ça avec MLflow.**

---

## Comment fonctionne ce notebook

Nous allons progressivement augmenter la difficulté :

| Partie | Style | Vous allez... |
|--------|-------|---------------|
| **Partie 1** | Entièrement guidé | Exécuter le code, observer, apprendre les concepts |
| **Partie 2** | Révélation progressive | Deviner ce qui est nécessaire, puis voir la solution |
| **Partie 3** | Compléter les blancs | Compléter le code vous-même |

**Référence** : Voir `02_mlflow_organized.ipynb` pour la solution complète.

**Cheatsheet** : Voir `../docs/mlflow_cheatsheet.md` pour une référence rapide.

---

## Prérequis

Avant de commencer, assurez-vous que :
1. Le serveur MLflow est en cours d'exécution : `docker-compose up -d` (depuis la racine du projet)
2. Vérifiez que http://localhost:5000 est accessible

## Setup : Charger les données du notebook chaotique

D'abord, reproduisons la même préparation de données que dans le notebook chaotique.

In [ ]:
# Imports standards (identiques au notebook chaotique)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

print("Imports terminés!")

In [ ]:
# Charger et préparer les données (identique au notebook chaotique)
df = pd.read_csv('../data/customer_data.csv')

# Feature engineering
df['recency_frequency_ratio'] = df['recency_days'] / (df['frequency'] + 1)
df['monetary_per_order'] = df['monetary_value'] / (df['total_orders'] + 1)
df['order_frequency'] = df['total_orders'] / (df['days_since_signup'] + 1)
df['support_per_order'] = df['support_tickets'] / (df['total_orders'] + 1)
df['r_score'] = pd.qcut(df['recency_days'], q=5, labels=[5, 4, 3, 2, 1]).astype(int)
df['f_score'] = pd.qcut(df['frequency'].rank(method='first'), q=5, labels=[1, 2, 3, 4, 5]).astype(int)
df['m_score'] = pd.qcut(df['monetary_value'].rank(method='first'), q=5, labels=[1, 2, 3, 4, 5]).astype(int)
df['rfm_score'] = df['r_score'] + df['f_score'] + df['m_score']

# Définir les features
FEATURE_COLS = [
    'recency_days', 'frequency', 'monetary_value', 'avg_order_value',
    'days_since_signup', 'total_orders', 'support_tickets', 'age',
    'recency_frequency_ratio', 'monetary_per_order', 'order_frequency',
    'support_per_order', 'rfm_score'
]

X = df[FEATURE_COLS]
y = df['churned']

# Séparation train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Données prêtes : {len(X_train)} train, {len(X_test)} test échantillons")
print(f"Features : {len(FEATURE_COLS)}")

---

# PARTIE 1 : Entièrement Guidé

## 1.1 Qu'est-ce que MLflow ?

MLflow est une plateforme open-source pour gérer le **cycle de vie complet du ML**. Elle a 4 composants principaux :

### 1. **Tracking** (notre focus principal aujourd'hui)
Enregistrer et requêter les expériences : paramètres, métriques, artefacts et versions du code.
- *"Quels hyperparamètres j'ai utilisé pour mon meilleur modèle ?"*
- *"Comparer l'accuracy sur 50 runs"*

### 2. **Models**
Packager les modèles ML dans un format standard pour le déploiement sur diverses plateformes (Docker, Kubernetes, services cloud, etc.).
- *"Déployer ce modèle sklearn comme API REST"*
- *"Convertir mon modèle pour de l'inférence batch"*

### 3. **Model Registry** (on va l'utiliser aussi)
Store centralisé de modèles avec versioning, transitions d'état (Staging → Production), et annotations.
- *"Promouvoir le modèle v3 en production"*
- *"Qui a approuvé cette version du modèle ?"*

### 4. **Projects**
Packager le code et les dépendances pour des exécutions reproductibles sur n'importe quelle plateforme.
- *"Exécuter cette expérience avec exactement le même environnement"*
- *"Partager mon pipeline avec l'équipe"*

---

### MLflow pour les LLMs (Bonus)

MLflow supporte maintenant les **applications LLM/GenAI** :
- **Tracing** : Débugger et monitorer les appels LLM, pipelines RAG, et agents multi-étapes
- **Évaluation** : Benchmarker les outputs LLM avec des métriques intégrées
- **Déploiement** : Servir les modèles LLM avec la même infrastructure de serving

*On ne couvre pas le tracking LLM aujourd'hui, mais sachez que MLflow fonctionne au-delà du ML traditionnel !*

---

**Aujourd'hui on se concentre sur Tracking et Registry** - les fondations pour la gestion d'expériences.

## 1.2 Se connecter à MLflow

D'abord, on importe MLflow et on lui indique où se trouve notre serveur de tracking.

In [ ]:
# NOUVEAU : Importer MLflow
import mlflow
import mlflow.sklearn

print(f"Version MLflow : {mlflow.__version__}")

In [ ]:
# Se connecter au serveur MLflow (qui tourne dans Docker)
mlflow.set_tracking_uri("http://localhost:5000")

# Vérifier la connexion
import requests
try:
    response = requests.get("http://localhost:5000/health")
    if response.status_code == 200:
        print("Le serveur MLflow fonctionne !")
    else:
        print(f"Le serveur a répondu avec : {response.status_code}")
except Exception as e:
    print(f"Impossible de se connecter au serveur MLflow : {e}")
    print("Exécutez 'docker-compose up -d' depuis la racine du projet !")

## 1.3 Créer une Expérience

Une **Expérience** regroupe les runs associés ensemble. Pensez-y comme un dossier de projet.

In [ ]:
# Créer ou récupérer une expérience
experiment_name = "workshop-churn-learning"
mlflow.set_experiment(experiment_name)

print(f"Utilisation de l'expérience : {experiment_name}")
print(f"Voir sur : http://localhost:5000")

## 1.4 Votre Premier Run MLflow

Un **Run** est une exécution unique de votre code d'entraînement. Dans un run, vous pouvez logger :

- **Parameters** : Les entrées (hyperparamètres, config)
- **Metrics** : Les sorties (accuracy, loss, F1)
- **Artifacts** : Les fichiers (modèles, graphiques, données)
- **Tags** : Les métadonnées (auteur, version)

In [ ]:
# Entraînons un modèle simple et LOGGONS TOUT

# Démarrer un run
with mlflow.start_run(run_name="mon-premier-run-mlflow"):
    
    # 1. LOGGER LES PARAMÈTRES (entrées de votre modèle)
    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", 10)
    
    # 2. Entraîner le modèle (même code qu'avant)
    model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    model.fit(X_train, y_train)
    
    # 3. Faire des prédictions
    y_pred = model.predict(X_test)
    
    # 4. LOGGER LES MÉTRIQUES (sorties/résultats)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("f1", f1)
    
    print(f"Accuracy : {accuracy:.4f}")
    print(f"Score F1 : {f1:.4f}")
    print("\nRun enregistré dans MLflow !")
    print("Allez sur http://localhost:5000 pour le voir !")

### Allez voir l'UI MLflow !

1. Ouvrez http://localhost:5000
2. Cliquez sur l'expérience "workshop-churn-learning"
3. Cliquez sur "mon-premier-run-mlflow"
4. Voyez vos paramètres et métriques !

**C'est déjà mieux que le notebook chaotique** - vos résultats sont sauvegardés et organisés !

## 1.5 Logger plusieurs métriques à la fois

Au lieu d'appeler `log_metric` plusieurs fois, vous pouvez logger un dictionnaire.

In [ ]:
# Plus efficace : logger plusieurs métriques d'un coup
with mlflow.start_run(run_name="exemple-metriques-multiples"):
    
    # Logger les params en dict
    mlflow.log_params({
        "model_type": "RandomForest",
        "n_estimators": 150,
        "max_depth": 12
    })
    
    # Entraîner
    model = RandomForestClassifier(n_estimators=150, max_depth=12, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Logger les métriques en dict
    mlflow.log_metrics({
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1": f1_score(y_test, y_pred)
    })
    
    print("Toutes les métriques loggées !")

---

# PARTIE 2 : Révélation Progressive

Maintenant vous comprenez les bases. Ajoutons plus de fonctionnalités MLflow.

**Format** : Je vous montre un problème, vous réfléchissez à ce qui est nécessaire, puis je révèle la solution.

## 2.1 Problème : Sauvegarder le modèle

Dans le notebook chaotique, on avait :
```python
# sauvegarder le modèle? je sais pas où
# import pickle
# with open('model.pkl', 'wb') as f:
#     pickle.dump(best_rf, f)
```

**Question** : Comment peut-on sauvegarder le modèle AVEC le run, pour qu'il soit toujours lié à ses métriques ?

*Réfléchissez-y... puis exécutez la cellule suivante pour voir la solution.*

In [ ]:
# SOLUTION : Utiliser mlflow.sklearn.log_model()

with mlflow.start_run(run_name="avec-artefact-modele"):
    
    mlflow.log_params({"n_estimators": 100, "max_depth": 10})
    
    model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mlflow.log_metrics({
        "accuracy": accuracy_score(y_test, y_pred),
        "f1": f1_score(y_test, y_pred)
    })
    
    # LOGGER LE MODÈLE - il est maintenant sauvegardé avec ce run !
    mlflow.sklearn.log_model(model, name="model")
    
    print("Modèle sauvegardé comme artefact !")
    print("Vérifiez l'onglet 'Artifacts' dans l'UI MLflow")

## 2.2 Problème : Sauvegarder les graphiques

Dans le notebook chaotique, on créait des matrices de confusion et des graphiques d'importance des features.
Mais ils s'affichaient juste dans le notebook - pas sauvegardés nulle part !

**Question** : Comment peut-on sauvegarder une figure matplotlib comme artefact ?

*Réfléchissez... puis révélez.*

In [ ]:
# SOLUTION : Utiliser mlflow.log_figure()

with mlflow.start_run(run_name="avec-graphiques"):
    
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mlflow.log_metrics({"accuracy": accuracy_score(y_test, y_pred)})
    
    # Créer le graphique de matrice de confusion
    cm = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
    ax.set_xlabel('Prédit')
    ax.set_ylabel('Réel')
    ax.set_title('Matrice de Confusion')
    
    # LOGGER LA FIGURE - pas de fichier local créé !
    mlflow.log_figure(fig, "matrice_confusion.png")
    plt.close()
    
    print("Graphique sauvegardé comme artefact !")

## 2.3 Problème : Le problème du Scaler

Pour la Régression Logistique, on doit normaliser les features :
```python
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
```

**Problème** : Au moment de l'inférence, on a besoin du MÊME scaler. Si on ne le sauvegarde pas, on ne peut pas faire de prédictions correctes plus tard !

**Question** : Comment sauvegarder le scaler avec le modèle ?

*Réfléchissez... puis révélez.*

In [ ]:
# SOLUTION : Sauvegarder le scaler comme artefact
import joblib
import tempfile
import os

with mlflow.start_run(run_name="logistique-avec-scaler"):
    
    # Normaliser les données
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Sauvegarder le scaler comme artefact (avec tempfile pour éviter les fichiers locaux)
    with tempfile.TemporaryDirectory() as tmpdir:
        scaler_path = os.path.join(tmpdir, "scaler.pkl")
        joblib.dump(scaler, scaler_path)
        mlflow.log_artifact(scaler_path, artifact_path="preprocessing")
    
    # Entraîner le modèle
    model = LogisticRegression(random_state=42, max_iter=1000)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    mlflow.log_metrics({"accuracy": accuracy_score(y_test, y_pred)})
    mlflow.sklearn.log_model(model, name="model")
    
    # Tag pour se rappeler que ce modèle nécessite un scaling
    mlflow.set_tag("requires_scaling", "true")
    
    print("Modèle + scaler sauvegardés !")
    print("Vérifiez les artefacts : preprocessing/scaler.pkl")

## 2.4 Problème : Comparer les runs

Dans le notebook chaotique :
```python
# ok j'abandonne d'essayer de suivre toutes ces expériences
# c'était lequel le meilleur? je crois que c'était celui de grid search?
```

**Question** : Comment trouver programmatiquement le meilleur run ?

*Réfléchissez... puis révélez.*

In [ ]:
# SOLUTION : Utiliser le Client MLflow pour rechercher les runs
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment = client.get_experiment_by_name(experiment_name)

# Rechercher les runs, triés par accuracy (décroissant)
runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["metrics.accuracy DESC"],
    max_results=5
)

print("Top 5 des runs par accuracy :")
print("=" * 60)
for run in runs:
    acc = run.data.metrics.get('accuracy', 0)
    f1 = run.data.metrics.get('f1', 0)
    print(f"{run.info.run_name}: accuracy={acc:.4f}, f1={f1:.4f}")

---

# PARTIE 3 : Compléter les blancs

Maintenant c'est à vous ! Complétez le code dans les cellules suivantes.

**Aide** : Référez-vous au cheatsheet dans `../docs/mlflow_cheatsheet.md`

## 3.1 Exercice : Logger un modèle Gradient Boosting

Complétez le code pour entraîner et logger un modèle Gradient Boosting avec :
- Paramètres : n_estimators, learning_rate, max_depth
- Métriques : accuracy, precision, recall, f1
- Artefact : le modèle entraîné

In [ ]:
# EXERCICE : Complétez ce code

with mlflow.start_run(run_name="exercice-gradient-boosting"):
    
    # TODO : Logger ces paramètres
    n_estimators = 100
    learning_rate = 0.1
    max_depth = 5
    
    # mlflow.log_params({...})  # <-- Complétez ceci
    
    # Entraîner le modèle
    model = GradientBoostingClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # TODO : Calculer et logger les métriques
    # mlflow.log_metrics({...})  # <-- Complétez ceci
    
    # TODO : Logger le modèle
    # mlflow.sklearn.log_model(...)  # <-- Complétez ceci
    
    print("Exercice terminé ! Vérifiez l'UI MLflow.")

<details>
<summary>Cliquez pour révéler la solution</summary>

```python
with mlflow.start_run(run_name="exercice-gradient-boosting"):
    
    n_estimators = 100
    learning_rate = 0.1
    max_depth = 5
    
    mlflow.log_params({
        "n_estimators": n_estimators,
        "learning_rate": learning_rate,
        "max_depth": max_depth
    })
    
    model = GradientBoostingClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mlflow.log_metrics({
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1": f1_score(y_test, y_pred)
    })
    
    mlflow.sklearn.log_model(model, name="model")
```

</details>

## 3.2 Exercice : Ajouter des tags et une matrice de confusion

Améliorez votre run avec :
- Tags : author (votre nom), model_type
- Un graphique de matrice de confusion comme artefact

In [ ]:
# EXERCICE : Complétez ce code

with mlflow.start_run(run_name="exercice-avec-tags-et-graphique"):
    
    # TODO : Ajouter des tags
    # mlflow.set_tag("author", "...")  # <-- Votre nom
    # mlflow.set_tag("model_type", "...")  # <-- Type de modèle
    
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
    
    # TODO : Créer et logger la matrice de confusion
    # cm = confusion_matrix(y_test, y_pred)
    # fig, ax = plt.subplots(...)
    # sns.heatmap(...)
    # mlflow.log_figure(...)  # <-- Complétez ceci
    # plt.close()
    
    print("Exercice terminé !")

## 3.3 Défi : Trouver et charger le meilleur modèle

Utilisez le Client MLflow pour :
1. Trouver le run avec la meilleure accuracy
2. Charger le modèle de ce run
3. Faire des prédictions sur de nouvelles données

In [ ]:
# DÉFI : Complétez ce code

from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment = client.get_experiment_by_name(experiment_name)

# TODO : Rechercher le meilleur run par accuracy
# best_runs = client.search_runs(
#     experiment_ids=[...],
#     order_by=[...],
#     max_results=1
# )

# if best_runs:
#     best_run = best_runs[0]
#     print(f"Meilleur run : {best_run.info.run_name}")
#     print(f"Accuracy : {best_run.data.metrics.get('accuracy', 0):.4f}")
    
#     # TODO : Charger le modèle
#     # model_uri = f"runs:/{best_run.info.run_id}/model"
#     # loaded_model = mlflow.sklearn.load_model(model_uri)
    
#     # TODO : Faire des prédictions
#     # predictions = loaded_model.predict(X_test[:5])
#     # print(f"Exemples de prédictions : {predictions}")

---

# PARTIE 4 : Serving & Inférence

Vous savez déjà servir des modèles avec joblib + FastAPI. Voyons comment MLflow simplifie cela.

## 4.1 Enregistrer un modèle

Avant de servir, enregistrons notre meilleur modèle dans le Model Registry.

In [ ]:
# D'abord, entraînons un modèle qu'on veut enregistrer
with mlflow.start_run(run_name="modele-pour-registry") as run:
    
    mlflow.log_params({"n_estimators": 200, "max_depth": 12})
    
    model = RandomForestClassifier(n_estimators=200, max_depth=12, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mlflow.log_metrics({
        "accuracy": accuracy_score(y_test, y_pred),
        "f1": f1_score(y_test, y_pred)
    })
    
    # Logger le modèle
    mlflow.sklearn.log_model(model, name="model")
    
    # Sauvegarder run_id pour l'enregistrement
    run_id = run.info.run_id
    print(f"Modèle loggé. Run ID : {run_id}")

In [ ]:
# Enregistrer le modèle dans le Model Registry
model_name = "churn-predictor"
model_uri = f"runs:/{run_id}/model"

# Enregistrer !
model_version = mlflow.register_model(model_uri, model_name)

print(f"Modèle enregistré : {model_name}")
print(f"Version : {model_version.version}")
print(f"\nVoir dans l'UI : http://localhost:5000/#/models/{model_name}")

## 4.2 Charger un modèle depuis le Registry

Au lieu de `joblib.load('model.pkl')`, vous pouvez charger par nom et version.

In [ ]:
# Charger le modèle depuis le registry - comparez avec ce que vous connaissez !

# ANCIENNE MÉTHODE (ce que vous connaissez déjà) :
# model = joblib.load('model.pkl')  # Où est-il ? Quelle version ?

# MÉTHODE MLFLOW :
# Charger la dernière version
loaded_model = mlflow.sklearn.load_model(f"models:/{model_name}/latest")

# Ou charger une version spécifique
# loaded_model = mlflow.sklearn.load_model(f"models:/{model_name}/1")

# Faire des prédictions - identique à avant !
sample_predictions = loaded_model.predict(X_test[:5])
print(f"Exemples de prédictions : {sample_predictions}")
print(f"\nModèle chargé depuis le registry : {model_name}/latest")

## 4.3 Servir le modèle comme API REST

Vous savez comment servir des modèles avec FastAPI :

```python
# ANCIENNE MÉTHODE - FastAPI + joblib
from fastapi import FastAPI
import joblib

app = FastAPI()
model = joblib.load('model.pkl')

@app.post("/predict")
def predict(data: dict):
    X = pd.DataFrame([data])
    return {"prediction": model.predict(X).tolist()}
```

**MLflow fait ça en une commande :**

```bash
mlflow models serve -m models:/churn-predictor/latest -p 5001 --no-conda
```

C'est tout. Pas besoin de code FastAPI.

In [ ]:
# Si vous exécutez la commande serve ci-dessus, vous pouvez l'appeler comme ceci :
# (C'est le même pattern que vous connaissez avec FastAPI !)

import requests

# Données exemple pour la prédiction
sample_data = X_test.head(3).to_dict(orient='split')

# Décommentez quand le serveur tourne :
# response = requests.post(
#     "http://localhost:5001/invocations",
#     json={"dataframe_split": sample_data}
# )
# print(response.json())

print("Pour tester le serving :")
print("1. Ouvrez un terminal")
print("2. Exécutez : mlflow models serve -m models:/churn-predictor/latest -p 5001 --no-conda")
print("3. Décommentez le code ci-dessus et exécutez cette cellule")

## 4.4 Inférence Batch (Charger depuis le Registry)

Pour des prédictions batch dans un pipeline, chargez et prédisez simplement :

In [ ]:
# Inférence batch - c'est ce que les orchestrateurs vont faire !

# Charger le modèle depuis le registry
model = mlflow.sklearn.load_model(f"models:/{model_name}/latest")

# Charger de nouvelles données (simuler un batch quotidien)
new_customers = df[FEATURE_COLS].head(100)

# Prédire
predictions = model.predict(new_customers)
probabilities = model.predict_proba(new_customers)[:, 1]

# Créer les résultats
results = pd.DataFrame({
    'customer_id': df['customer_id'].head(100),
    'churn_probability': probabilities,
    'churn_predicted': predictions
})

print(f"Prédictions batch pour {len(results)} clients :")
print(results.head(10))

## 4.5 Comparaison : Ce que vous connaissez vs MLflow

| Tâche | Manuel (joblib + FastAPI) | MLflow |
|-------|---------------------------|--------|
| Sauvegarder modèle | `joblib.dump(model, 'model_v2.pkl')` | `mlflow.sklearn.log_model(model, name="model")` |
| Charger modèle | `joblib.load('model_v2.pkl')` | `mlflow.sklearn.load_model("models:/name/1")` |
| Versioning | Noms de fichiers manuels (`model_v1.pkl`, `model_v2.pkl`) | Versions automatiques (1, 2, 3...) |
| Servir en API | Écrire du code FastAPI, lancer uvicorn | `mlflow models serve -m models:/name/1` |
| Tracker quel modèle | Espérer se souvenir / vérifier les dates de fichiers | Registry montre version, métriques, qui l'a entraîné |
| Rollback | Trouver l'ancien fichier, espérer qu'il marche | `mlflow.sklearn.load_model("models:/name/1")` |

**MLflow ne remplace pas vos compétences** - il les enrichit avec le versioning, le tracking et l'automatisation.

---

# Résumé

## Ce que vous avez appris

| Concept | Code |
|---------|------|
| Se connecter à MLflow | `mlflow.set_tracking_uri("http://localhost:5000")` |
| Créer une expérience | `mlflow.set_experiment("name")` |
| Démarrer un run | `with mlflow.start_run():` |
| Logger des paramètres | `mlflow.log_params({"key": value})` |
| Logger des métriques | `mlflow.log_metrics({"accuracy": 0.95})` |
| Logger un modèle | `mlflow.sklearn.log_model(model, name="model")` |
| Logger une figure | `mlflow.log_figure(fig, "plot.png")` |
| Logger un artefact | `mlflow.log_artifact("file.pkl")` |
| Définir des tags | `mlflow.set_tag("author", "me")` |
| Rechercher des runs | `client.search_runs(...)` |
| Enregistrer un modèle | `mlflow.register_model(uri, "name")` |
| Charger depuis le registry | `mlflow.sklearn.load_model("models:/name/latest")` |
| Servir un modèle | `mlflow models serve -m models:/name/1 -p 5001` |

## Le Pipeline ML Complet

```
┌──────────────┐    ┌──────────────┐    ┌──────────────┐    ┌──────────────┐
│  DÉVELOPPER  │ →  │  ENTRAÎNER   │ →  │  ENREGISTRER │ →  │    SERVIR    │
├──────────────┤    ├──────────────┤    ├──────────────┤    ├──────────────┤
│  Notebooks   │    │ Orchestrateur│    │    Model     │    │  API REST    │
│ Expériences  │    │  + Tracking  │    │   Registry   │    │   ou Batch   │
└──────────────┘    └──────────────┘    └──────────────┘    └──────────────┘
    Partie 1-3         Partie 4           Partie 4           Partie 4
```

## Prochaines étapes

**Notebook de référence** : Voir `02_mlflow_organized.ipynb` pour la version complète, prête pour la production.

**Orchestrateurs** : Maintenant que vous comprenez MLflow, automatisons ce pipeline !
- **Prefect** : Voir `pipelines/Prefect_ML_Pipeline.py` - entraîne ET fait de l'inférence
- **Dagster** : Voir `pipelines/Dagster_ML_Pipeline.py` - approche centrée sur les assets
- **Airflow** : Voir `pipelines/Airflow_ML_Pipeline.py` - standard de l'industrie

Les orchestrateurs vont :
1. Planifier des runs d'entraînement quotidiens
2. Logger tout dans MLflow automatiquement
3. Enregistrer les nouvelles versions de modèles
4. Exécuter l'inférence batch et sauvegarder les prédictions

---